In [3]:
from methods.llm import get_async_vllm_endpoint
import os 

# Unlimited LLM endpoints
endpoint_id = "vllm-8sz1f7zg7oy0ui"
api_key = "rpa_EPOJED42G59S80Y6SKMCOI330EQU4JPPMKV2UD2W7j0uku"
get_endpoint_response = get_async_vllm_endpoint(endpoint_id, api_key)

#### Temasek Foundation Dataset

In [10]:
# 1. Test cases (dataloading ...)
import sys 
sys.path.append("../notebook/")
from optm.soft_prompt import load_tf_data

train_data, test_data = load_tf_data("../data/processed_data_clean.json")

# reference metric fuction below ... we care about prediction of label, specifically "no"
# we care about precision more than we care abou recall on 'No' prediction 

# Node dataset should ideally be concise and pure application info included ...
from methods.meta_prompt import MetaPrompt, PromptMode

tf_meta_prompt = MetaPrompt(
    task = "Evaluate grant application, make a decision (Yes, No, Maybe) and a brief comment explanating your decision on why this project is likely to be accepted or rejected.",
    func_name = "evaluate_grant",
    inputs = ["project_description"],
    outputs = ["label", "comment"],
    input_types = ["str"],
    output_types = ["str", "str"],
    mode = PromptMode.PROMPT
)

# Prepare test cases :: input dict & output dict
test_cases = []
for prompt, label, comment in zip(train_data["prompt"], train_data["label"], train_data["comment"]):
    test_cases.append(({"project_description": prompt}, {"label": label, "comment": comment}))
test_cases = test_cases[:3]

    
# Custom metric for TF datase

from typing import Union, Callable


def tf_label_metric(target_labels: Union[str, list], pred_labels: Union[str, list],
                    beta: float = 5):
    """ 
    "No" prediction > "Yes" prediction 
    Recall > Precision
    Weighted score (recall + precision) with beta : 1 as weights 
    """
    if isinstance(target_labels, str):
        target_labels = [target_labels]
    if isinstance(pred_labels, str): 
        pred_labels = [pred_labels]
        
    # Convert labels to lowercase for case-insensitive comparison
    target_labels = [label.lower() for label in target_labels]
    pred_labels = [label.lower() for label in pred_labels]
    
    # For single label comparison, we can simplify:
    # true positive: target is "no" and prediction is "no"
    true_pos = sum(1 for t, p in zip(target_labels, pred_labels) if t == "no" and p == "no")
    
    # Calculate precision and recall
    # If target is "no", recall denominator is 1, else 0
    # If pred is "no", precision denominator is 1, else 0
    no_targets = sum(1 for t in target_labels if t == "no")
    no_preds = sum(1 for p in pred_labels if p == "no")
    
    no_precision = true_pos / no_preds if no_preds > 0 else 0
    no_recall = true_pos / no_targets if no_targets > 0 else 0
    
    # Simple weighted score using beta
    weighted_score = ((1 / (1 + beta)) * no_precision + (beta / (1 + beta)) * no_recall) if (no_precision + no_recall) > 0 else 0.0
    
    err_msg = []
    for t, p in zip(target_labels, pred_labels):
        if t != p: 
            err_msg.append(f"Target is '{t}', but prediction is '{p}'") 
        
    return weighted_score, "\n".join(err_msg) if err_msg else ""



tf_metric_map = {"label": tf_label_metric}
    

# Specific Metric required for TF dataset (Worth refactor the code a bit)
from methods.llm import get_groq_response
from methods.evolnode import EvolNode 
node = EvolNode(tf_meta_prompt, None, None, get_response=get_groq_response, test_cases=test_cases,
                custom_metric_map=tf_metric_map) # setting manual test cases


In [14]:
node.evolve("i1", batch_size=3)


ValueError: Metric not defined for type: <class 'str'>, use LLM-based alignment check instead !

tf_meta_prompt

In [2]:
from methods.meta_prompt import MetaPrompt, PromptMode
from methods.evolnode import EvolNode
from methods.llm import get_groq_response, get_claude_response

# Code + Compilor Task
# mp = MetaPrompt("Search for age of a celebrity.", "get_celeb_age", ["name"], ["age"], ["str"], ["int"], PromptMode.CODE)
# Prompt + LLM Task
mp = MetaPrompt("Get the age of a celebrity.", "get_celeb_age", ["name"], ["age"], ["str"], ["int"], PromptMode.PROMPT) # 

test_cases = [
    ({"name": "Dilireba"}, {"age": 32}),
    ({"name": "ChengXiao"}, {"age": 26})
]

test_inputs = [c[0] for c in test_cases]

node = EvolNode(mp, None, None, get_response=get_endpoint_response, test_cases=test_cases) # setting manual test cases

node.evolve("i1", replace=True, batch_size=20, num_runs=2, print_summary=True) # Scale up batch size


input_dict = {"name": "Dilireba"}
output_dict = node(input_dict, max_attempts=6) # Batch Inference with vLLM

# node.get_response = get_groq_response # fast sequential inference
# output_dict = node(input_dict, max_attempts=6, batch_inference=False)
 
print("Output dict: ", output_dict)

Processing LLM queries: 100%|██████████| 20/20 [00:10<00:00,  1.97it/s]


 :: Total time elapsed: 10.15s, 0 errors


Processing LLM queries: 100%|██████████| 29/29 [00:25<00:00,  1.13it/s]


 :: Total time elapsed: 25.63s, 0 errors


Processing LLM queries: 0it [00:00, ?it/s]


 :: Total time elapsed: 0.00s, 0 errors
🏆 Best Code Performance Summary 🏆
  ⚡ Structural fitness: 0.33
  🎯 Functional fitness: 1.00
  ⭐ Global fitness:     0.67
  🔄 Compiled solutions:        17
  ⏱️ Time breakdown:
     :: Query time: 33.43s
     :: Evolution time: 10.19s
     :: Evaluation time: 55.46s
     :: Total time: 99.09s


📊 Code 0: Fitness: 0.0%
--------------------------------------------------------------------------------
❌ Error Messages:
name 're' is not defined
name 're' is not defined


📊 Code 1: Fitness: 0.0%
--------------------------------------------------------------------------------
❌ Error Messages:
Failed to parse LLM response -- JsonDecodeError : 
Expecting value: line 1 column 1 (char 0)AstLiteralError : 
invalid syntax (<unknown>, line 1)
Failed to parse LLM response -- JsonDecodeError : 
Expecting value: line 1 column 1 (char 0)AstLiteralError : 
invalid syntax (<unknown>, line 1)


📊 Code 2: Fitness: 66.7%


📊 Code 3: Fitness: 66.7%


📊 Code 4: Fitness: 

Processing LLM queries: 100%|██████████| 6/6 [00:10<00:00,  1.74s/it]

 :: Total time elapsed: 10.45s, 0 errors
Output dict:  {'age': 32}
